# This notebook has the code for After Merging part of the two datasets

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

## Load and process two different datasets

In [2]:
# Load datasets
energy_df = pd.read_csv("data/energy_dataset.csv")
weather_df = pd.read_csv("data/weather_features.csv")

## Process Energery Dataset
energy_df['time'] = pd.to_datetime(energy_df['time'], utc=True, infer_datetime_format=True)
energy_df = energy_df.set_index('time')
energy_df.interpolate(method='linear', limit_direction='forward', inplace=True, axis=0)

## Process weather dataset
def df_convert_dtypes(df, convert_from, convert_to):
    cols = df.select_dtypes(include=[convert_from]).columns
    for col in cols:
        df[col] = df[col].values.astype(convert_to)
    return df

weather_df = df_convert_dtypes(weather_df, np.int64, np.float64)
# Convert dt_iso to datetime type, rename it and set it as index
weather_df ['time'] = pd.to_datetime(weather_df ['dt_iso'], utc=True, infer_datetime_format=True)
weather_df  = weather_df.drop(['dt_iso'], axis=1)
weather_df = weather_df.set_index('time')
cities = weather_df['city_name'].unique()
# Create df_weather_2 and drop duplicate rows in df_weather
df_weather_2 = weather_df.reset_index().drop_duplicates(subset=['time', 'city_name'], 
                                                        keep='last').set_index('time')
df_weather = weather_df.reset_index().drop_duplicates(subset=['time', 'city_name'],
                                                      keep='first').set_index('time')
# Drop columns with qualitative weather information
df_weather = df_weather.drop(['weather_main', 'weather_id', 
                              'weather_description', 'weather_icon'], axis=1)

## Merging Two datasets

In [3]:
## Merging two datasets

# Split the df_weather into 5 dataframes (one for each city)
df_1, df_2, df_3, df_4, df_5 = [x for _, x in df_weather.groupby('city_name')]
dfs = [df_1, df_2, df_3, df_4, df_5]

# Merge all dataframes into the final dataframe
df_final = energy_df
for df in dfs:
    city = df['city_name'].unique()
    city_str = str(city).replace("'", "").replace('[', '').replace(']', '').replace(' ', '')
    df = df.add_suffix('_{}'.format(city_str))
    df_final = df_final.merge(df, on=['time'], how='outer')
    df_final = df_final.drop('city_name_{}'.format(city_str), axis=1)
merged_df = df_final.copy()

# Processing day, month, hour
def split_date(df):
    df['Date']= pd.to_datetime(df.index)
    df["Year"]= df.Date.dt.year
    df["Month"] = df.Date.dt.month
    df["Day"] = df.Date.dt.day
    df["Hour"] = df.Date.dt.hour
    df['WeekOfYear'] = df.Date.dt.isocalendar().week
split_date(merged_df)

In [4]:
merged_df.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,rain_1h_Valencia,rain_3h_Valencia,snow_3h_Valencia,clouds_all_Valencia,Date,Year,Month,Day,Hour,WeekOfYear
time,,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,2014-12-31 23:00:00+00:00,2014,12,31,23,1
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,2015-01-01 00:00:00+00:00,2015,1,1,0,1
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,2015-01-01 01:00:00+00:00,2015,1,1,1,1
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,2015-01-01 02:00:00+00:00,2015,1,1,2,1
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,2015-01-01 03:00:00+00:00,2015,1,1,3,1


## Features Selection

In [5]:
inputs = ['price actual', 'temp_Barcelona',
       'temp_min_Barcelona', 'temp_max_Barcelona', 'pressure_Barcelona',
       'humidity_Barcelona', 'wind_speed_Barcelona', 'wind_deg_Barcelona',
       'rain_1h_Barcelona',
       'clouds_all_Barcelona', 'temp_Bilbao', 'temp_min_Bilbao',
       'temp_max_Bilbao', 'pressure_Bilbao', 'humidity_Bilbao',
       'wind_speed_Bilbao', 'wind_deg_Bilbao', 'rain_1h_Bilbao',
        'clouds_all_Bilbao', 'temp_Madrid',
       'temp_min_Madrid', 'temp_max_Madrid', 'pressure_Madrid',
       'humidity_Madrid', 'wind_speed_Madrid', 'wind_deg_Madrid',
       'rain_1h_Madrid',
       'clouds_all_Madrid', 'temp_Seville', 'temp_min_Seville',
       'temp_max_Seville', 'pressure_Seville', 'humidity_Seville',
       'wind_speed_Seville', 'wind_deg_Seville', 'rain_1h_Seville',
       'clouds_all_Seville',
       'temp_Valencia', 'temp_min_Valencia', 'temp_max_Valencia',
       'pressure_Valencia', 'humidity_Valencia', 'wind_speed_Valencia',
       'wind_deg_Valencia', 'rain_1h_Valencia', 
       'clouds_all_Valencia','Day','Month','Year','Hour']
target = 'total load actual'

selected_df = merged_df[inputs+[target]].copy()
selected_df.reset_index(drop=True,inplace=True)
selected_df.head()

,price actual,temp_Barcelona,temp_min_Barcelona,temp_max_Barcelona,pressure_Barcelona,humidity_Barcelona,wind_speed_Barcelona,wind_deg_Barcelona,rain_1h_Barcelona,clouds_all_Barcelona,...,humidity_Valencia,wind_speed_Valencia,wind_deg_Valencia,rain_1h_Valencia,clouds_all_Valencia,Day,Month,Year,Hour,total load actual
0,65.41,281.625,281.625,281.625,1035.0,100.0,7.0,58.0,0.0,0.0,...,77.0,1.0,62.0,0.0,0.0,31,12,2014,23,25385.0
1,64.92,281.625,281.625,281.625,1035.0,100.0,7.0,58.0,0.0,0.0,...,77.0,1.0,62.0,0.0,0.0,1,1,2015,0,24382.0
2,64.48,281.286,281.286,281.286,1036.0,100.0,7.0,48.0,0.0,0.0,...,78.0,0.0,23.0,0.0,0.0,1,1,2015,1,22734.0
3,59.32,281.286,281.286,281.286,1036.0,100.0,7.0,48.0,0.0,0.0,...,78.0,0.0,23.0,0.0,0.0,1,1,2015,2,21286.0
4,56.04,281.286,281.286,281.286,1036.0,100.0,7.0,48.0,0.0,0.0,...,78.0,0.0,23.0,0.0,0.0,1,1,2015,3,20264.0


In [6]:
correlations = selected_df.corr(method='pearson')
print(correlations['total load actual'].sort_values(ascending=False).to_string())

total load actual       1.000000
price actual            0.435253
Hour                    0.395586
temp_min_Valencia       0.235297
temp_Valencia           0.220790
temp_min_Seville        0.206488
temp_max_Madrid         0.204718
temp_Seville            0.204571
temp_max_Valencia       0.199833
temp_Bilbao             0.196535
temp_max_Bilbao         0.194181
temp_min_Bilbao         0.193706
temp_min_Barcelona      0.187417
temp_Madrid             0.185388
temp_max_Seville        0.176234
temp_Barcelona          0.167338
wind_speed_Bilbao       0.155898
temp_min_Madrid         0.153753
wind_speed_Valencia     0.147982
temp_max_Barcelona      0.146444
wind_speed_Seville      0.132973
wind_speed_Barcelona    0.120719
wind_speed_Madrid       0.083669
Year                    0.060122
wind_deg_Bilbao         0.050107
rain_1h_Madrid          0.047272
clouds_all_Valencia     0.040361
rain_1h_Valencia        0.029616
rain_1h_Bilbao          0.028971
clouds_all_Madrid       0.022172
pressure_V

## Splitting datasets into train, validation, and test sets

In [7]:
# Splitting datasets into train, validation and test sets
train_set = selected_df[selected_df["Year"]<2018].copy()
test_set = selected_df[(selected_df["Year"]==2018) & (selected_df["Month"]<=6)].copy()
val_set = selected_df[(selected_df["Year"]==2018) & (selected_df["Month"]>6)].copy()

# Scaling the columns from 0 to 1 limit
scaler = MinMaxScaler().fit(train_set[inputs])
train_set[inputs] = scaler.transform(train_set[inputs])
val_set[inputs] = scaler.transform(val_set[inputs])
test_set[inputs] = scaler.transform(test_set[inputs])

# Setting X and y for train, test, validation
[X_train, y_train] = [train_set[inputs].copy(), train_set[target].copy()]
[X_val, y_val] = [val_set[inputs].copy(), val_set[target].copy()]
[X_test, y_test] = [test_set[inputs].copy(), test_set[target].copy()]

## Training and Validating the model

In [8]:
def checking_error(model):
    train_preds = model.predict(X_train)
    train_err = mean_squared_error(train_preds, y_train, squared=False)
    val_preds = model.predict(X_val)
    val_err = mean_squared_error(val_preds, y_val, squared=False)
    print(f'Training error: {train_err} ;  Validation error: {val_err}')

In [9]:
# Training the model
model = XGBRegressor(random_state=42, n_jobs=-1, n_estimators=10, max_depth=3, subsample=0.9, 
                     colsample_bytree=0.7, seed =1301)
model.fit(X_train, y_train)
checking_error(model)

Training error: 2908.621256906358 ;  Validation error: 2990.775095091549
